# PH 245 Homework 3: Logistic Regression
Kunal Mishra

In [154]:
# Loading Libraries
library(ggplot2)
library(dummies)

# Loading Data
data = read.table(file="Data-HW3-CHeartDisease.dat", header=FALSE, quote="", sep=",")

# Processing data and removing patients with missing data 
length = nrow(data)
id.ms = sort( c(seq(1, length)[data[,12]=='?'],
                seq(1, length)[data[,13]=='?']
               )
            )

data[,12] = as.numeric(data[,12]) - 2
data[,13] = as.numeric(data[,13]) - 1

predictors = data.matrix(data[,1:13])
response = data[,14]
response[response > 0] = 1

colnames(predictors) = c("age", "gender", "chestpain", "bldpressure", "chol",
      "bldsugar", "electrocardio", "heartrate", "angina", "STdepression",
      "STslope", "vessel", "thal")

# Removing patients without valid data
predictors = predictors[-id.ms,]
response = response[-id.ms]

stopifnot(nrow(predictors) == length(response) && nrow(predictors) == 297)

print("Predictors:")
head(predictors)

print("Response:")
head(response)

[1] "Predictors:"


age,gender,chestpain,bldpressure,chol,bldsugar,electrocardio,heartrate,angina,STdepression,STslope,vessel,thal
63,1,1,145,233,1,2,150,0,2.3,3,0,2
67,1,4,160,286,0,2,108,1,1.5,2,3,1
67,1,4,120,229,0,2,129,1,2.6,2,2,3
37,1,3,130,250,0,0,187,0,3.5,3,0,1
41,0,2,130,204,0,2,172,0,1.4,1,0,1
56,1,2,120,236,0,0,178,0,0.8,1,0,1


[1] "Response:"


[1] 0 1 1 0 0 0

#### Problem 1A

In [155]:
# Exploratory Data Analysis 

# 1.A.1: How many patients in the dataset had heart disease vs. no disease? 
numHeartDisease = sum(response)
noHeartDisease = length(response) - numHeartDisease

print("Total number of patients:")
length(response)
stopifnot(length(response) == numHeartDisease + noHeartDisease)

print("Number of patients with heart disease:")
numHeartDisease

print("Number of patients with no heart disease:")
noHeartDisease

"________________________________________________________________"
# 1.A.2: Which predictors are numerical, which are categorical, and which are unclear? 

print("Total number of predictors:")
ncol(predictors)

print("Numeric predictor variables:")
head(predictors[,c(1, 4, 5, 8, 10)])


print("Categorical predictor variables:")
head(predictors[,c(2, 3, 6, 9, 12, 13)])

print("Unclear variables that could be treated as either numeric or categorical:")
head(predictors[, c(7, 11)])

"________________________________________________________________"
# Further EDA
print("Gender breakdown % (0):")
(297-sum(predictors[,2]))/297 * 100

print("Gender breakdown % (1):")
sum(predictors[,2])/297 * 100

[1] "Total number of patients:"


[1] 297

[1] "Number of patients with heart disease:"


[1] 137

[1] "Number of patients with no heart disease:"


...

[1] "________________________________________________________________"

[1] "Gender breakdown % (0):"


[1] 32.32323

[1] "Gender breakdown % (1):"


[1] 67.67677

One thing that isn't completely clear is what each category for the gender variable is. Based on the fact that heart disease is a more prevalent problem for men and 67% of our dataset is labeled as "1" in terms of gender, it is fairly likely that the mapping is 0=Female, 1=Male

#### Problem 1B

In [157]:
combinedDF = as.data.frame(cbind(predictors, response))
fit = glm(formula=response~., family="binomial", data=combinedDF)
summary(fit)


Call:
glm(formula = response ~ ., family = "binomial", data = combinedDF)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.8042  -0.5263  -0.1860   0.4161   2.3676  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -6.993701   2.893938  -2.417  0.01566 *  
age           -0.014057   0.024036  -0.585  0.55866    
gender         1.319688   0.486718   2.711  0.00670 ** 
chestpain      0.578582   0.191335   3.024  0.00250 ** 
bldpressure    0.024182   0.010727   2.254  0.02418 *  
chol           0.004816   0.003775   1.276  0.20202    
bldsugar      -0.991868   0.554947  -1.787  0.07389 .  
electrocardio  0.246117   0.185238   1.329  0.18396    
heartrate     -0.021183   0.010275  -2.062  0.03923 *  
angina         0.915651   0.414003   2.212  0.02699 *  
STdepression   0.249909   0.212418   1.176  0.23940    
STslope        0.582699   0.362317   1.608  0.10778    
vessel         1.267008   0.265723   4.768 1.86e-06 ***
thal        

#### Problem 1C

To begin, we'll transform two of our categorical variables -- chest pain type and thal type -- using a one-hot encoding.

In [169]:
combinedOneHotDF = dummy.data.frame(combinedDF, names=c("chestpain", "thal"))

# Dropping one of each new dummy columns to avoid linear dependence/multicollinearity ("Dummy Variable Trap")
combinedOneHotDF$chestpain1 = NULL
combinedOneHotDF$thal1 = NULL


# Refit the logistic regression to include the new dummy variables
oneHotFit = glm(formula=response~., family="binomial", data=combinedOneHotDF)

summary(oneHotFit)


Call:
glm(formula = response ~ ., family = "binomial", data = combinedOneHotDF)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.7145  -0.5436  -0.1444   0.3264   2.7316  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -5.986093   2.938058  -2.037 0.041607 *  
age           -0.012296   0.024664  -0.499 0.618120    
gender         1.431422   0.513185   2.789 0.005282 ** 
chestpain2     1.071153   0.753902   1.421 0.155371    
chestpain3     0.202175   0.648718   0.312 0.755304    
chestpain4     2.006802   0.652608   3.075 0.002105 ** 
bldpressure    0.023981   0.011110   2.159 0.030889 *  
chol           0.004930   0.003944   1.250 0.211306    
bldsugar      -0.610758   0.599184  -1.019 0.308052    
electrocardio  0.255433   0.189565   1.347 0.177829    
heartrate     -0.021281   0.010821  -1.967 0.049224 *  
angina         0.739431   0.434687   1.701 0.088931 .  
STdepression   0.353095   0.230102   1.535 0.124903    
STslop

#### Problem 1D

The coefficient estimate for serum cholesterol was .00493. This can be interpreted as a .00493 increase in the log odds of having heart disease compared to not having heart disease for every unit increase in blood serum cholesterol. Though a variable like this would seem to intuitively be significant for predicting heart disease, the p-value for seeing a coefficient estimate as extreme as this purely due to chance is .2113. With an alpha of .05, we fail to reject the null hypothesis that blood serum cholesterol == 0.

#### Problem 1E

The coefficient estimate for chest pain type 4 was 2.006802. This can be interpreted as people having this chest pain see a 2.006802 increase in the log odds of having heart disease compared to those having chest pain type 1, which was excluded from our analysis to preseve linear independence. The p-value of chestpain4 was .002105, indicating that we could reject our null hypothesis (alpha=.05) that chestpain4's coefficient == 0. Our results indicate that the coefficient estimate is indeed statistically significant to the model and predicting heart disease outcomes.

#### Problem 1F

In order to generate binary predictions from our model, we'll need to classify based on the probability predicted by our model. 

In [194]:
probabilityPredictions = as.numeric(predict(oneHotFit, combinedOneHotDF, type='response'))

print("Reminder: 0=Heart Disease Absent; 1=Heart Disease Present")
""

print("Head of Probability Predictions (%Chance that response was not 0)")
head(probabilityPredictions)

binaryResponsePredictions = as.numeric(probabilityPredictions >= .5)

print("Head of Response Predictions based on model")
head(binaryResponsePredictions)

print("Head of True responses for training set")
head(combinedOneHotDF$response)

stopifnot(length(binaryResponsePredictions) == length(combinedOneHotDF$response))

accuracy = sum(binaryResponsePredictions == combinedOneHotDF$response)/length(binaryResponsePredictions)
print("Model accuracy:")
accuracy

print("Misclassification rate:")
1-accuracy


[1] "Reminder: 0=Heart Disease Absent; 1=Heart Disease Present"


[1] ""

[1] "Head of Probability Predictions (%Chance that response was not 0)"


[1] 0.23862882 0.99850760 0.99540963 0.23335447 0.03625226 0.04899566

[1] "Head of Response Predictions based on model"


...

[1] 0 1 1 0 0 0

[1] "Model accuracy:"


[1] 0.8619529

[1] "Misclassification rate:"


[1] 0.1380471